In [8]:
pip install scikit-learn

     ---------------------------------------- 11.0/11.0 MB 1.2 MB/s eta 0:00:00
     -------------------------------------- 301.8/301.8 kB 1.2 MB/s eta 0:00:00
     ---------------------------------------- 46.2/46.2 MB 1.2 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.2.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [9]:
import os
import cv2
import json
import numpy as np
import pandas as pd
from urllib.parse import urlparse
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

In [10]:
# Load and preprocess the dataset

def parse_url(url):
    parsed_url = urlparse(url)
    domain = parsed_url.netloc
    return f"{domain}.jpg"

def load_data():
    # Implement loading and preprocessing your dataset 
   
    f = open('lighthouse_report.json')
    data = json.load(f)['lighthouseReports']
    f.close

    ratings = pd.DataFrame(data)
    
    ratings['filepath'] = ratings['url'].apply(parse_url)
    ratings.drop('id', axis=1, inplace=True)
        
    images = []
    for i in range(0,len(ratings)):
        filepath = ratings['filepath'][i]
        images.append(cv2.imread('images/{0}'.format(filepath),0))
    
    return np.array(images), np.array(ratings['seo'])

In [25]:
images, ratings = load_data()
images = images / 255.0  # normalize images

print(images.shape)

X_train, X_test, y_train, y_test = train_test_split(images, ratings, test_size=0.2, random_state=42)

print(X_test)


(43, 224, 224)


In [24]:

# Define the model
model = Sequential([
    Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(224, 224, 1)),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.25),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.25),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='linear')  # Output layer for regression
])

model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])
model.summary()

# Train the model
history = model.fit(X_train, y_train, epochs=20, validation_split=0.2, batch_size=32)


# Evaluate the model
test_loss, test_mae = model.evaluate(X_test, y_test)
print(f'Test MAE: {test_mae}')

# Predict rating for a new screenshot
def predict_rating(screenshot):
    screenshot = screenshot / 255.0
    screenshot = np.expand_dims(screenshot, axis=0)  # Add batch dimension
    predicted_rating = model.predict(screenshot)
    return predicted_rating[0][0]

# Example usage
new_screenshot = cv2.imread('images/{0}'.format('hydromotor.pl.jpg'),0)  # Load your new screenshot
predicted_rating = predict_rating(new_screenshot)
print(f'Predicted Rating: {predicted_rating}')

Model: "sequential_11"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_22 (Conv2D)              │ (None, 222, 222, 32)   │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_22 (MaxPooling2D) │ (None, 111, 111, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_15 (Dropout)            │ (None, 111, 111, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_23 (Conv2D)              │ (None, 109, 109, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_23 (MaxPooling2D) │ (None, 54, 54, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_16 (Dropout)            │ (None, 54, 54, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_11 (Flatten)            │ (None, 186624)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_22 (Dense)                │ (None, 128)            │    23,888,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_17 (Dropout)            │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_23 (Dense)                │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 23,906,945 (91.20 MB)

 Trainable params: 23,906,945 (91.20 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - loss: 8001.5908 - mae: 89.0368 - val_loss: 6626.1025 - val_mae: 80.6874
Epoch 2/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 850ms/step - loss: 7829.8442 - mae: 88.0705 - val_loss: 6320.9780 - val_mae: 78.7742
Epoch 3/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 815ms/step - loss: 7322.6221 - mae: 85.1427 - val_loss: 5705.6958 - val_mae: 74.7669
Epoch 4/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 893ms/step - loss: 6411.1787 - mae: 79.5962 - val_loss: 4683.7007 - val_mae: 67.5880
Epoch 5/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 781ms/step - loss: 5210.0020 - mae: 71.6317 - val_loss: 3288.9871 - val_mae: 56.3338
Epoch 6/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 783ms/step - loss: 3461.6182 - mae: 58.1541 - val_loss: 1700.4159 - val_mae: 39.8123
Epoch 7/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 862ms/step - loss: 1540.3479 - mae: 37.2265 - val_loss: 398.2371 - val_mae: 18.6627
Epoch 8/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 900ms/step - loss: 329.7287 - mae: 14.5917 - val_loss: 250.1824 - val_mae: 11.6168
Epoch 9/20
1/1